In [23]:
import numpy as np
import pandas as pd
from vtk import *
from vtk.util import numpy_support
import imp
import h5py
import matplotlib.pylab as plt
%matplotlib tk

from sklearn.neighbors import KDTree
import os
from tqdm import tqdm

In [24]:
from experiments import MeshLoad

In [25]:
FILE_MESH = './test3.vtk'

In [26]:
ml = MeshLoad(FILE_MESH, mesh_type='tri')
ml.update()
print ml.cells_array, ml.points_array

[[ 228  605  607]
 [ 605  606  607]
 [ 605  262  606]
 ..., 
 [2016  984 2013]
 [2016  373  984]
 [2013  984  261]] [[ 0.          0.        ]
 [ 1.          0.        ]
 [ 1.          1.        ]
 ..., 
 [ 0.48091934  0.4676754 ]
 [ 0.47870318  0.44509473]
 [ 0.46405484  0.45793146]]


In [27]:
# (A+R)c = b+r

In [28]:
class Tri:
    
    def __init__(self, p1, p2, p3):
        self.x1 = np.matrix(p1).transpose()
        self.x2 = np.matrix(p2).transpose()
        self.x3 = np.matrix(p3).transpose()
        self.diff = np.matrix(np.eye(2))
        
    def J(self):
        return np.vstack([np.ones(3), np.hstack([self.x1, self.x2, self.x3])])
    
    def det_J(self):
        return np.linalg.det(self.J())
    
    def inv_J(self):
        return np.linalg.inv(self.J())
    
    def V(self):
        return self.det_J()/2
        
    def det_J(self):
        return np.linalg.det(self.J())
    
    def grad_N(self):
        
        ai = self.inv_J()[:,1]
        bi = self.inv_J()[:,2]
        
        return np.hstack([ai,bi]).transpose()
        
    def A(self):
        ret = np.matrix(np.zeros((3,3)))
        t_ = self.grad_N()
        
        for i in xrange(3):
            for j in xrange(3):
                ret[i,j] = (t_[0,i]*t_[0,j] + t_[1,i]*t_[1,j])*self.V()
        
        return ret

In [29]:
idx = 10

p1 = ml.points_array[ml.cells_array[10,0]]
p2 = ml.points_array[ml.cells_array[10,2]]
p3 = ml.points_array[ml.cells_array[10,1]]

tri = Tri(p1,p2,p3)
tri.grad_N()

matrix([[ 44.86648715, -43.12981771,  -1.73666945],
        [ 45.34460476,  -2.63707858, -42.70752618]])

In [30]:
tri.A()

matrix([[ 1.10731502, -0.5591254 , -0.54818962],
        [-0.5591254 ,  0.50809499,  0.05103041],
        [-0.54818962,  0.05103041,  0.49715921]])

In [31]:
matrix_size = len(ml.points_array)

In [32]:
full_A = np.matrix(np.zeros((matrix_size,matrix_size)))

for elem in ml.cells_array:
    a,b,c = elem
    
    p1 = ml.points_array[a]
    p2 = ml.points_array[c]
    p3 = ml.points_array[b]

    tri = Tri(p1,p2,p3)
    assert(tri.det_J() > 0)
    for i in xrange(3):
        for j in xrange(3):
            full_A[elem[i], elem[j]] += tri.A()[i,j]

In [33]:
plt.imshow(full_A, interpolation='none')

In [34]:
full_A

matrix([[ 0.99878255,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.99878255,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.99878255, ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  3.73375452,
         -0.52803047, -0.71655437],
        [ 0.        ,  0.        ,  0.        , ..., -0.52803047,
          3.35670672, -0.52803047],
        [ 0.        ,  0.        ,  0.        , ..., -0.71655437,
         -0.52803047,  3.54523062]])

In [35]:
full_b = np.zeros((matrix_size,))
full_b[51] = 1
full_b[501] = -1

In [36]:
lhs = full_A#+full_R
rhs = full_b

full_x = np.linalg.solve(lhs,rhs)

In [37]:
ml.write_result(full_x)